In [1]:
import sys
sys.path.insert(0, "../")

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from mcfa import (mcfa, mpl_utils)

%matplotlib inline

In [ ]:
# Load the Iris data.
X = load_iris().data

In [ ]:
# Plot the data as a corner plot.
fig = mpl_utils.corner_scatter(X, s=10, alpha=0.75, figsize=(10, 10))

In [ ]:
model = mcfa.MCFA(n_components=3, n_latent_factors=3, verbose=1)
model.fit(X)

In [ ]:
for parameter_name, value in dict(zip(model.parameter_names, model.theta_)).items():
    print("{}: {}\n{}".format(parameter_name, value.shape, value))

In [ ]:
# Plot the estimated posterior factor scores.
#fig = mpl_utils.corner_scatter(X, s=10, alpha=0.75, figsize=(10, 10))
v, v_cluster, v_mean = model.factor_scores(X)

# Associations
hard_associations = np.argmax(model.tau_, axis=1)

fig = mpl_utils.corner_scatter(v_mean, c=hard_associations, figsize=(10, 10), s=10)

In [ ]:
fig = model.plot_latent_space(X, s=10, figsize=(10, 10))

In [ ]:
# Plot the clustering in data space.
fig = mpl_utils.corner_scatter(X, s=10, alpha=0.75, figsize=(10, 10), c=hard_associations)

In [ ]:
# Plot the factor loads.
A = model.theta_[model.parameter_names.index("A")]
fig, ax = plt.subplots()
ax.scatter(A.T[0], A.T[1])
model.theta_[1].shape


In [ ]:
fig, ax = plt.subplots()
for i in range(model.n_latent_factors):
    ax.plot(A.T[i])

psi = model.theta_[model.parameter_names.index("psi")]
ax.plot(psi, c="#666666")